In [2]:
import pandas as pd
import numpy as np
import os
import glob

from google.cloud import storage

from tqdm import tqdm

import cv2
from PIL import Image

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf
from tensorflow.data import Dataset

2024-03-10 21:06:44.675359: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-10 21:06:44.717492: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-10 21:06:44.717527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-10 21:06:44.718536: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-10 21:06:44.724174: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-10 21:06:44.725276: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [18]:
BUCKET_NAME = os.environ.get('BUCKET_NAME')
BUCKET_NAME

'gs://mammo_data'

In [15]:
# Initialiser le client GCS
# client = storage.Client(project='mammo-analysis')
# bucket = client.bucket('mammo_data')

# Créer un dossier "clean_data" dans le bucket "mammo_data"
# if 'clean_data' not in list(client.list_buckets()):
#     blob = bucket.blob('clean_data/')
#     blob.upload_from_string('')
    # CLEAN_DATA = list(client.list_buckets())['clean_data']

# Lister les buckets d'un projet GCP
# buckets = list(client.list_buckets())
# bucket_names = [bucket.name for bucket in buckets]
# bucket_names


# Lister les blobs (objet) d'un bucket GCP
# blobs = list(bucket.list_blobs())
# blob_names = [blob.name for blob in blobs]
# if 'clean_data/' in blob_names:
#     print('True')
# else:
#     print('False')

True


In [20]:
CUR_DIR = os.getcwd()
DRAFT_DIR = os.path.dirname(CUR_DIR)
PROJECT_DIR = os.path.dirname(DRAFT_DIR)
DATA_DIR = os.path.join(PROJECT_DIR,'data')
METADATA_DIR = os.path.join(DATA_DIR, 'metadata')
IMAGE_DIR = os.path.join(DATA_DIR,'images')
PROCESSED_DIR = os.path.join(IMAGE_DIR,'data_processed')
DATA_CLEAN = os.path.join(IMAGE_DIR, 'clean')
file = 'kaggle.xlsx'

DIM = os.environ.get('DIM')

In [22]:
df = pd.read_excel(os.path.join(METADATA_DIR, file))
df.head()
df.shape

(54706, 14)

In [4]:
df_train = df[['image_id', 'cancer']].sort_values(by='image_id')
df_train

,image_id,cancer
18887,68491,0
21242,83977,0
13448,127964,0
22857,193459,0
827,225973,0
...,...,...
36145,2147266534,0
53694,2147293172,0
23227,2147355367,0
42946,2147415608,0


In [6]:
def load_image(path:str, dim=DIM, nb_img=None, extension='.jpg'):
    '''
    Load, Resize, Convert into np.Array all images (.jpg) from a path and Save new images into a "path/clean" folder
    '''
    for file in tqdm(os.listdir(path), ascii=True, desc="load|resize|convert|save"):
        if file.endswith(extension):
            img = cv2.imread(os.path.join(path, file))
            resize_img = tf.image.resize(img, DIM, method='nearest')
            file_name = f"{os.path.splitext(file)[0]}{extension}"
            if not os.path.exists(DATA_CLEAN):
                os.makedirs(DATA_CLEAN)
            cv2.imwrite(os.path.join(DATA_CLEAN, file_name), resize_img)
            print(f'Image {file_name} saved !')

load_image(PROCESSED_DIR)

load|resize|convert|save:   2%|2         | 2/100 [00:00<00:05, 16.39it/s]

**Attention** Image 1834201085.jpg saved ! **Attention**
**Attention** Image 227919373.jpg saved ! **Attention**
**Attention** Image 2048218442.jpg saved ! **Attention**


load|resize|convert|save:   6%|6         | 6/100 [00:00<00:08, 11.46it/s]

**Attention** Image 1808945924.jpg saved ! **Attention**
**Attention** Image 1458259706.jpg saved ! **Attention**
**Attention** Image 136943557.jpg saved ! **Attention**


load|resize|convert|save:   8%|8         | 8/100 [00:00<00:07, 12.35it/s]

...

**Attention** Image 1322198700.jpg saved ! **Attention**
**Attention** Image 364263039.jpg saved ! **Attention**


load|resize|convert|save:  92%|#########2| 92/100 [00:06<00:00, 12.97it/s]

**Attention** Image 2069718338.jpg saved ! **Attention**
**Attention** Image 1488169873.jpg saved ! **Attention**
**Attention** Image 389308620.jpg saved ! **Attention**
**Attention** Image 2109485229.jpg saved ! **Attention**


load|resize|convert|save: 100%|##########| 100/100 [00:07<00:00, 14.03it/s]

**Attention** Image 825330094.jpg saved ! **Attention**
**Attention** Image 2095534993.jpg saved ! **Attention**
**Attention** Image 2053681232.jpg saved ! **Attention**
**Attention** Image 473043625.jpg saved ! **Attention**
**Attention** Image 249024760.jpg saved ! **Attention**
**Attention** Image 886468142.jpg saved ! **Attention**


In [7]:
files = glob.glob(os.path.join(DATA_CLEAN,'*.jpg'))
files, len(files)

(['/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/1834201085.jpg',
  '/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/227919373.jpg',
  '/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/2048218442.jpg',
  '/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/1808945924.jpg',
  '/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/1458259706.jpg',
  '/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/136943557.jpg',
  '/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/633991369.jpg',
  '/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/178764132.jpg',
  '/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/1854424773.jpg',
  '/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/291057023.jpg',
  '/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/140750770.jpg',
  '/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/458523517.jpg',
  '/home/remy/code/Lalucarne51/mamm

In [8]:
sample_ids = []
for f in files:
    sample_ids.append(int((f.split('/')[-1].split(".")[0])))

sorted_sample_ids = sorted(sample_ids)
len(sorted_sample_ids)

100

In [9]:
df_train = df_train[df_train['image_id'].isin(sample_ids)]
df_train

,image_id,cancer
41324,9531619,0
6460,40744983,1
22026,64688863,0
7164,136943557,0
53883,140750770,0
...,...,...
16207,2095534993,0
24329,2109485229,0
3005,2110177776,0
54695,2129119612,0


In [10]:
df_train['cancer'].value_counts()

cancer
0    98
1     2
Name: count, dtype: int64

In [11]:
paths = []
for id in sorted_sample_ids:
    paths.append(f"{DATA_CLEAN}/{id}.jpg")
paths[0:4]

['/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/9531619.jpg',
 '/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/40744983.jpg',
 '/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/64688863.jpg',
 '/home/remy/code/Lalucarne51/mammo-analysis/data/images/clean/136943557.jpg']

In [12]:
df_train['paths'] = paths
df_train = df_train.reset_index().drop('index', axis=1)

/tmp/ipykernel_32783/229523953.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['paths'] = paths


In [13]:
df_train.head()

,image_id,cancer,paths
0,9531619,0,/home/remy/code/Lalucarne51/mammo-analysis/dat...
1,40744983,1,/home/remy/code/Lalucarne51/mammo-analysis/dat...
2,64688863,0,/home/remy/code/Lalucarne51/mammo-analysis/dat...
3,136943557,0,/home/remy/code/Lalucarne51/mammo-analysis/dat...
4,140750770,0,/home/remy/code/Lalucarne51/mammo-analysis/dat...


In [14]:
df_train.shape

(100, 3)

In [29]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image_id  100 non-null    int64 
 1   cancer    100 non-null    int64 
 2   paths     100 non-null    object
dtypes: int64(2), object(1)
memory usage: 2.5+ KB


In [54]:
labels = (df_train['cancer'])
type(labels)

pandas.core.series.Series

In [55]:
file_names = df_train['paths']
type(file_names)

pandas.core.series.Series

In [52]:
dataset = dataset.map(lambda file_name, label: (load_image(file_name), label))

TypeError: in user code:

    File "/tmp/ipykernel_32783/3014967802.py", line 1, in None  *
        lambda file_name, label: (load_image(file_name), label)
    File "/tmp/ipykernel_32783/3285479193.py", line 5, in load_image  *
        for file in tqdm(os.listdir(path)[0:nb_img], ascii=True, desc="load|resize|convert|save"):

    TypeError: listdir: path should be string, bytes, os.PathLike, integer or None, not SymbolicTensor


In [49]:
dataset = Dataset.from_tensor_slices((file_names, labels))
dataset

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [40]:
def initialize_model():
    model = models.Sequential()
    model.add(layers.Conv2D(16, (4, 4), input_shape = (128, 128, 1), activation = 'relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation = 'relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(units = 10, activation = 'relu'))
    model.add(layers.Dense(units = 1, activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy',
                    optimizer='adam',
                    metrics =['recall'])
    return model

In [50]:
model = initialize_model()
es = EarlyStopping(patience = 5)
history = model.fit(dataset,
                    validation_split = 0.2,
                    batch_size = 32,
                    epochs = 5,
                    callbacks = [es],
                    verbose = 1)

ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset'>]

In [11]:
from google.cloud import storage

# Initialiser le client GCS
client = storage.Client(project="mammo-analysis")
buckets = list(client.list_buckets())

# Afficher les noms des buckets
for bucket in buckets:
    print(bucket.name)

# def read_csv():
#     '''
#     Make a CSV ready to be used
#     '''
#     pass


Forbidden: 403 GET https://storage.googleapis.com/storage/v1/b?project=mammo-analysis&projection=noAcl&prettyPrint=false: gcp-service-account@engaged-bivouac-411320.iam.gserviceaccount.com does not have storage.buckets.list access to the Google Cloud project. Permission 'storage.buckets.list' denied on resource (or it may not exist).

In [1]:
def load_image(path:str):
    '''
    Make a Dataset of Tensors images from a CSV
    '''

    variables = pd.read_csv(os.path.join(DATA_DIR, 'final.csv'), index_col= None, header=[0])
    resized_images = []

    for idx, image_id, image_jpeg_path, image_array_path, label in variables.values:
        # Read and Resize images
        img = tf.io.read_file(PROCESSED_DIR)
        img = tf.io.decode_image(img)
        img = tf.image.resize(img, [DIM, DIM], method='nearest')
        resized_images.append(img)
        # Convert labels to Tensor
        labels_tensor = tf.cast(labels, tf.int32)
        labels_tensor = tf.constant(labels_tensor, shape=(1667, 1, 1))

        # Convert images to Tensor
        image_tensor = tf.cast(resized_images, dtype= tf.float32) / 255
        image_tensor = tf.constant(image_tensor, shape=(1667, 1, DIM, DIM, 1))

    # Construct Dataset from tensors
    dataset = tf.data.Dataset.from_tensor_slices((image_tensor, labels_tensor))

    return dataset

In [28]:
dataset = dataset.map(lambda file_name, label: (load_image(file_name), label))

TypeError: in user code:

    File "/tmp/ipykernel_32783/3014967802.py", line 1, in None  *
        lambda file_name, label: (load_image(file_name), label)
    File "/tmp/ipykernel_32783/3285479193.py", line 5, in load_image  *
        for file in tqdm(os.listdir(path)[0:nb_img], ascii=True, desc="load|resize|convert|save"):

    TypeError: listdir: path should be string, bytes, os.PathLike, integer or None, not SymbolicTensor


In [22]:
for element in dataset:
    print(element.shape)

AttributeError: 'tuple' object has no attribute 'shape'